# Comprehensive SQL Demo in ColabThis notebook demonstrates core SQL concepts using Python's built-in `sqlite3` module. It is self-contained and uses an in-memory SQLite database so it runs without any external dependencies.

## 1. Connect to SQLite and Define Helper UtilitiesWe start by establishing a connection to an in-memory SQLite database. We also define a helper function to execute SQL statements and neatly display results.

In [ ]:
import sqlite3
from textwrap import dedent

conn = sqlite3.connect(":memory:")
conn.row_factory = sqlite3.Row

print("SQLite version:", sqlite3.sqlite_version)

def run_query(sql, params=None, fetch=True, many=False):
    """Execute SQL and print results if requested."""
    if params is None:
        params = []
    with conn:
        cur = conn.cursor()
        if many:
            cur.executemany(sql, params)
        else:
            cur.execute(sql, params)
        if fetch:
            rows = cur.fetchall()
            if not rows:
                print("No rows returned.")
                return rows
            headers = rows[0].keys()
            widths = [max(len(str(h)), max(len(str(row[h])) for row in rows)) for h in headers]
            header_row = " | ".join(f"{h:<{w}}" for h, w in zip(headers, widths))
            print(header_row)
            print("-+-".join("-" * w for w in widths))
            for row in rows:
                print(" | ".join(f"{str(row[h]):<{w}}" for h, w in zip(headers, widths)))
        return cur


## 2. Create Tables and Load Sample DataWe build a small school-like schema with `students`, `courses`, `enrollments`, and `instructors`. The helper function runs multiple insert statements inside a transaction.

In [ ]:
setup_sql = dedent('''
    CREATE TABLE instructors (
        instructor_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department TEXT NOT NULL
    );

    CREATE TABLE courses (
        course_id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        department TEXT NOT NULL,
        credits INTEGER NOT NULL,
        instructor_id INTEGER REFERENCES instructors(instructor_id)
    );

    CREATE TABLE students (
        student_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        enrollment_year INTEGER NOT NULL,
        major TEXT NOT NULL
    );

    CREATE TABLE enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER REFERENCES students(student_id),
        course_id INTEGER REFERENCES courses(course_id),
        grade TEXT,
        UNIQUE(student_id, course_id)
    );
''')

with conn:
    conn.executescript(setup_sql)

instructors = [
    (1, 'Dr. Smith', 'Computer Science'),
    (2, 'Dr. Lee', 'Mathematics'),
    (3, 'Dr. Patel', 'Physics')
]

courses = [
    (1, 'Data Structures', 'Computer Science', 4, 1),
    (2, 'Algorithms', 'Computer Science', 4, 1),
    (3, 'Calculus I', 'Mathematics', 3, 2),
    (4, 'Linear Algebra', 'Mathematics', 3, 2),
    (5, 'Classical Mechanics', 'Physics', 4, 3)
]

students = [
    (1, 'Alice', 'Johnson', 2021, 'Computer Science'),
    (2, 'Bob', 'Smith', 2020, 'Mathematics'),
    (3, 'Carlos', 'Diaz', 2021, 'Physics'),
    (4, 'Diana', 'Lee', 2019, 'Computer Science'),
    (5, 'Emily', 'Wang', 2022, 'Mathematics')
]

enrollments = [
    (1, 1, 1, 'A'),
    (2, 1, 2, 'B'),
    (3, 2, 3, 'A'),
    (4, 2, 4, 'B'),
    (5, 3, 5, 'A'),
    (6, 4, 1, 'A'),
    (7, 4, 2, 'A'),
    (8, 5, 3, 'B'),
    (9, 5, 4, 'A')
]

run_query('INSERT INTO instructors VALUES (?, ?, ?)', instructors, fetch=False, many=True)
run_query('INSERT INTO courses VALUES (?, ?, ?, ?, ?)', courses, fetch=False, many=True)
run_query('INSERT INTO students VALUES (?, ?, ?, ?, ?)', students, fetch=False, many=True)
run_query('INSERT INTO enrollments VALUES (?, ?, ?, ?)', enrollments, fetch=False, many=True)

print('Sample data inserted successfully.')


## 3. Basic `SELECT` QueriesRetrieve all students and list specific columns to illustrate projection and ordering.

In [ ]:
run_query('SELECT student_id, first_name, last_name, major FROM students ORDER BY student_id')


## 4. Filtering with `WHERE`Apply conditions using comparison and logical operators.

In [ ]:
run_query(
    '''
    SELECT first_name, last_name, enrollment_year, major
    FROM students
    WHERE major = ? AND enrollment_year >= ?
    ORDER BY enrollment_year
    ''',
    ('Computer Science', 2020)
)


## 5. Calculated Columns and AliasesCreate computed values inside `SELECT` statements and rename columns with aliases for readability.

In [ ]:
run_query(
    '''
    SELECT
        first_name || ' ' || last_name AS full_name,
        enrollment_year,
        2024 - enrollment_year AS years_in_program
    FROM students
    ORDER BY years_in_program DESC
    '''
)


## 6. Aggregations with `GROUP BY`Use aggregate functions like `COUNT`, `AVG`, and `SUM` alongside grouping and `HAVING` clauses.

In [ ]:
run_query(
    '''
    SELECT major, COUNT(*) AS student_count
    FROM students
    GROUP BY major
    HAVING COUNT(*) >= 1
    ORDER BY student_count DESC
    '''
)


## 7. Joining TablesCombine data across multiple tables using `JOIN`. This example shows enrolled courses per student.

In [ ]:
run_query(
    '''
    SELECT s.first_name || ' ' || s.last_name AS student,
           c.title AS course,
           c.department,
           e.grade
    FROM enrollments AS e
    JOIN students AS s ON s.student_id = e.student_id
    JOIN courses AS c ON c.course_id = e.course_id
    ORDER BY s.last_name, c.title
    '''
)


## 8. Multi-Table Joins with Additional ContextIllustrate joining instructors to courses and their enrolled students.

In [ ]:
run_query(
    '''
    SELECT c.title,
           i.name AS instructor,
           COUNT(e.student_id) AS enrollment_count
    FROM courses AS c
    JOIN instructors AS i ON i.instructor_id = c.instructor_id
    LEFT JOIN enrollments AS e ON e.course_id = c.course_id
    GROUP BY c.course_id
    ORDER BY enrollment_count DESC
    '''
)


## 9. SubqueriesUse a subquery to find students whose enrollment count exceeds the average.

In [ ]:
run_query(
    '''
    SELECT s.first_name || ' ' || s.last_name AS student,
           COUNT(e.course_id) AS course_load
    FROM students AS s
    JOIN enrollments AS e ON e.student_id = s.student_id
    GROUP BY s.student_id
    HAVING COUNT(e.course_id) > (
        SELECT AVG(course_total)
        FROM (
            SELECT COUNT(*) AS course_total
            FROM enrollments
            GROUP BY student_id
        )
    )
    ORDER BY course_load DESC
    '''
)


## 10. Data Manipulation (`INSERT`, `UPDATE`, `DELETE`)Show how to add a new student, update a grade, and remove an enrollment, all while checking results.

In [ ]:
# INSERT a new student and enrollment
run_query(
    'INSERT INTO students (student_id, first_name, last_name, enrollment_year, major) VALUES (?, ?, ?, ?, ?)',
    (6, 'Farah', 'Nguyen', 2023, 'Physics'),
    fetch=False
)
run_query(
    'INSERT INTO enrollments (enrollment_id, student_id, course_id, grade) VALUES (?, ?, ?, ?)',
    (10, 6, 5, 'B'),
    fetch=False
)

print('After insert:')
run_query(
    '''
    SELECT s.first_name || ' ' || s.last_name AS student, c.title, e.grade
    FROM enrollments AS e
    JOIN students AS s ON s.student_id = e.student_id
    JOIN courses AS c ON c.course_id = e.course_id
    WHERE s.student_id = ?
    ''',
    (6,)
)

# UPDATE grade
run_query(
    'UPDATE enrollments SET grade = ? WHERE enrollment_id = ?',
    ('A', 10),
    fetch=False
)
print('After update:')
run_query(
    'SELECT grade FROM enrollments WHERE enrollment_id = ?',
    (10,)
)

# DELETE enrollment
run_query(
    'DELETE FROM enrollments WHERE enrollment_id = ?',
    (10,),
    fetch=False
)
print('After delete:')
run_query(
    'SELECT * FROM enrollments WHERE enrollment_id = ?',
    (10,)
)


## 11. TransactionsWrap statements in an explicit transaction and demonstrate rollback vs. commit behavior.

In [ ]:
try:
    conn.execute('BEGIN')
    conn.execute('INSERT INTO courses (course_id, title, department, credits, instructor_id) VALUES (?, ?, ?, ?, ?)',
                 (6, 'Quantum Computing', 'Computer Science', 4, 1))
    conn.execute('INSERT INTO courses (course_id, title, department, credits, instructor_id) VALUES (?, ?, ?, ?, ?)',
                 (6, 'Duplicate Course', 'Computer Science', 3, 1))
    conn.commit()
except sqlite3.IntegrityError as exc:
    print('IntegrityError encountered:', exc)
    conn.rollback()

print('Courses after rollback attempt:')
run_query('SELECT course_id, title FROM courses ORDER BY course_id')


## 12. ViewsCreate a view to simplify repeated queries and show how to use it.

In [ ]:
run_query('DROP VIEW IF EXISTS student_gpa_view', fetch=False)
run_query(
    '''
    CREATE VIEW student_gpa_view AS
    SELECT s.student_id,
           s.first_name || ' ' || s.last_name AS student,
           s.major,
           AVG(CASE e.grade
                   WHEN 'A' THEN 4.0
                   WHEN 'B' THEN 3.0
                   WHEN 'C' THEN 2.0
                   WHEN 'D' THEN 1.0
                   ELSE 0.0
               END) AS gpa
    FROM students AS s
    LEFT JOIN enrollments AS e ON e.student_id = s.student_id
    GROUP BY s.student_id
    ''',
    fetch=False
)

print('Student GPA view:')
run_query('SELECT * FROM student_gpa_view ORDER BY gpa DESC')


## 13. Window Functions (SQLite 3.25+)Demonstrate ranking students by GPA using window functions, which are supported in modern SQLite versions.

In [ ]:
run_query(
    '''
    SELECT student,
           major,
           gpa,
           RANK() OVER (ORDER BY gpa DESC) AS gpa_rank
    FROM student_gpa_view
    '''
)


## 14. Cleaning UpClose the database connection when finished. Colab will clean resources automatically, but it's good practice.

In [ ]:
conn.close()
print('Connection closed.')
